In [ ]:
pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 879.0/879.0 kB 11.2 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)

# Train the model
results = model.train(data="/content/drive/MyDrive/project8/wound.yaml", epochs=100, imgsz=640 ,
                      project="/content/drive/MyDrive/project8/model_results")

In [2]:
from ultralytics import YOLO
import torch
import os
import json
from PIL import Image , ImageDraw
import numpy as np
import cv2


# Load YOLOv8 models with pretrained weights
model_a = YOLO("/content/drive/MyDrive/project8/model_results/train3/weights/last.pt")  # initial detection
model_b = YOLO("/content/drive/MyDrive/project8/lastsegcategory.pt")  # Second model for segmentation

input_dir = "/content/drive/MyDrive/project8/model_results/input_test"
output_dir = "/content/drive/MyDrive/project8/model_results/output_test"
os.makedirs(output_dir, exist_ok=True)  # Ensure output directory exists

def pipeline_yolo_models(input_dir):
    for image_name in os.listdir(input_dir):
        image_path = os.path.join(input_dir, image_name)

        # Ensure the file is an image
        if image_path.lower().endswith(('.jpg', '.jpeg', '.png')):
            # Step 1: Load the original image
            original_image = cv2.imread(image_path)

            # Step 2: Run Model A for initial detections
            results_a = model_a.predict(source=image_path, save=False)

            # Optionally save the Model A output image with detections drawn
            model_a_output_path = os.path.join(output_dir, f"detected_{image_name}")
            results_a[0].plot()  # Generate a plot of the results
            results_a[0].save(model_a_output_path)  # Save the output image with detections

            # Step 3: Run Model B on the output of Model A (saved image)
            results_b = model_b.predict(source=model_a_output_path, save=False)  # Don't save automatically

            # Optionally save the final output image from Model B
            model_b_output_path = os.path.join(output_dir, f"segmented_{image_name}")
            results_b[0].save(model_b_output_path)  # Save the output image from Model B

            # print(f"Saved detection output for {image_name} at {model_a_output_path}")
            # print(f"Saved segmentation output for {image_name} at {model_b_output_path}")
            os.remove(model_a_output_path)

# Run the pipeline
pipeline_yolo_models(input_dir)




image 1/1 /content/drive/MyDrive/project8/model_results/input_test/cut (20).jpg: 640x640 3 Cutss, 8.8ms
Speed: 2.0ms preprocess, 8.8ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/drive/MyDrive/project8/model_results/output_test/detected_cut (20).jpg: 640x640 1 acute, 20.8ms
Speed: 1.6ms preprocess, 20.8ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/drive/MyDrive/project8/model_results/input_test/burns (16).jpg: 640x640 4 Burnss, 8.4ms
Speed: 1.8ms preprocess, 8.4ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/drive/MyDrive/project8/model_results/output_test/detected_burns (16).jpg: 640x640 (no detections), 20.8ms
Speed: 2.0ms preprocess, 20.8ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/drive/MyDrive/project8/model_results/input_test/burns (2).jpg: 640x640 1 Burns, 7.5ms
Speed: 1.9ms preprocess, 7.5ms inference, 1.3ms postp

In [ ]:
from ultralytics import YOLO
import cv2
import os

# Load YOLOv8 models with pretrained weights
model_a = YOLO("/content/drive/MyDrive/project8/model_results/train3/weights/last.pt")  # initial detection
model_b = YOLO("/content/drive/MyDrive/project8/lastsegcategory.pt")  # Second model for segmentation

output_dir = "/content/drive/MyDrive/project8/model_results/output_test"
os.makedirs(output_dir, exist_ok=True)  # Ensure output directory exists

def pipeline_yolo_camera():
    # Open the default camera (index 0)
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Error: Could not open camera.")
        return

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Failed to capture image.")
            break

        # Step 1: Run Model A for initial detections
        results_a = model_a.predict(source=frame, save=False)

        # Draw the Model A results on the frame
        frame_with_detections = results_a[0].plot()  # Annotate frame with detection results

        # Step 2: Run Model B for segmentation on Model A's output
        results_b = model_b.predict(source=frame, save=False)

        # Draw the Model B results (segmentation) on the frame
        frame_with_segmentation = results_b[0].plot()

        # Display the final output
        cv2.imshow("YOLO Detection and Segmentation", frame_with_segmentation)

        # Press 'q' to exit the loop
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the capture and close any OpenCV windows
    cap.release()
    cv2.destroyAllWindows()

# Run the pipeline
pipeline_yolo_camera()

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Error: Could not open camera.
